# Testing LeNet-5
A tutorial to test LeNet-5 using ADAPT. Before start tutorial, if you use your GPU, the following cell will set tensorflow to use minimal memory.

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

## Load model to test
Here, we use LeNet-5 and we offers how to create a LeNet-5 model.

### 1. Load MNIST
You can easily load MNIST data supplyied by Tensorflow/Keras.

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

Normalize the values of each pixel into 0 and 1, and convert the label to categorical vector.

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 2. Create model
The structure of the model is originally from the following paper:

    Gradient-based learning applied to document recognition

In [ ]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Model

In [ ]:
def LeNet5():
    
    # Input layer.
    input_tensor = Input(shape=(28, 28, 1))
    
    # Block 1.
    x = Convolution2D(6, (5, 5), activation='relu', padding='same', name='block1_conv1')(input_tensor)
    x = MaxPooling2D(pool_size=(2, 2), name='block1_pool1')(x)
    
    # Block 2.
    x = Convolution2D(16, (5, 5), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D(pool_size=(2, 2), name='block2_pool1')(x)
    
    # Fully connected.
    x = Flatten(name='flatten')(x)
    x = Dense(120, activation='relu', name='fc1')(x)
    x = Dense(84, activation='relu', name='fc2')(x)
    x = Dense(10, name='before_softmax')(x)
    x = Activation('softmax', name='redictions')(x)
    
    return Model(input_tensor, x)

In [ ]:
model = LeNet5()
model.summary()

### 3. Train model
Train the model with the adam optimizer for 10 epochs.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_ = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=256, epochs=10, verbose=1)

The final accuracy of the trained model is about 99%.

## Test using ADAPT
From now on, let's test the created model.

### 1. Choose candidate input
In this tutorial, 20 randomly chosen images from the test set will be used.
Here is how they look.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
indices = [800, 3433, 3496, 3592, 4322, 4395, 4706, 6101, 6223, 6715, 7190, 7302, 7491, 7715, 8149, 8267, 8608, 9197, 9721, 9795]
images = x_test[indices]
fig, ax = plt.subplots(1, len(images), figsize=(len(images) * 2, 2))
for i, idx in enumerate(indices):
    ax[i].set_axis_off()
    ax[i].title.set_text(str(np.argmax(y_test[idx])))
    ax[i].imshow(np.reshape(images[i], (28, 28)), cmap='gray')
plt.show()

### 2. Create a fuzzer
ADAPT offers various modules (e.g. coverage metrics and neuron selection strategies) that can be used to compose a fuzzer. First thing to do is wrapping Keras model, since all modules in ADAPT uses a Keras model wrapped with the ```adapt.Network``` class.

In [ ]:
from adapt import Network

In [ ]:
network = Network(model)

Create a neuron coverage with 0.5 as threshold.

In [ ]:
from adapt.metric import NC

In [ ]:
metric = NC(0.5)

Create an adaptive and parameterized neuron selection strategy introduced in the following paper:

    Effective White-box Testing of Deep Neural Networks with Adaptive Neuron-Selection Strategy
    
Note that the selection strategies and the fuzzers are created for each indivisual input.

In [ ]:
from adapt.fuzzer import WhiteBoxFuzzer
from adapt.strategy import AdaptiveParameterizedStrategy

In [ ]:
fuzzers = []
for image in images:
    strategy = AdaptiveParameterizedStrategy(network)
    fuzzer = WhiteBoxFuzzer(network, image, metric, strategy)
    fuzzers.append(fuzzer)

### 3. Start testing
The given inputs are tested for 1 minute (20 minutes in total), and only keep the generated inputs with minimum distances.

In [ ]:
from datetime import datetime

In [ ]:
print('Fuzzing started at {}.'.format(datetime.now()))
archives = [fuzzer.start(minutes=1, append='min_dist') for fuzzer in fuzzers]
print('Done')

### 4. Testing result
Here is the coverage graph.

In [ ]:
times = list(range(61))
coverages = []
for archive in archives:
    current_t = 0
    coverage = []
    for t, cov in archive.timestamp:
        if t > current_t:
            coverage.append(cov)
            current_t += 1
    coverages.append(coverage)
coverages = np.mean(coverages, axis=0)
plt.plot(times, coverages)
plt.show()

Archive classes stores all the testing result. You can easily see the testing result by following command.

In [ ]:
archives[0].summary()

Plus, the following is a visualization of some generated images.

In [ ]:
fig, ax = plt.subplots(1, len(archives[0].found_labels), figsize=(len(archives[0].found_labels) * 2, 2))
for i, label in enumerate(archives[0].found_labels.keys()):
    ax[i].set_axis_off()
    ax[i].title.set_text(str(label))
    ax[i].imshow(np.reshape(archives[0].inputs[label][0], (28, 28)), cmap='gray')
plt.show()